# Självstudieövningar – Vecka 6

De här övningarna är till för att du ska repetera och befästa det vi har gått igenom under veckan samt introducera några nya begrepp på ett tryggt sätt.

Det innebär att vissa av uppgifterna kan du behöva söka efter information själv hur du löser. Här kan du använda Google eller AI om du vill men dokumentationerna är din bästa vän när du lär dig.
- Python: https://docs.python.org/3.13/
- Matplotlib: https://matplotlib.org/stable/
- Pandas: https://pandas.pydata.org/
- NumPy: https://numpy.org/

Läs först igenom vad som ska göras i respektive uppgift och fyll sedan i kodblocket nedanför, eller gör uppgiften i en separat fil.

**Detta lämnas inte in för rättning.** Arbeta i din egen takt, testa dig fram, använd dokumentationen när du kör fast och ta med eventuella frågor till nästa lektionstillfälle.

## (Kör först) Skapa veckans dataset

In [ ]:
import numpy as np
import pandas as pd

rng = np.random.default_rng(123)

n = 800
cities = np.array(["Göteborg","Malmö","Stockholm","Uppsala"])
genders = np.array(["F","M"])
groups = np.array(["A","B"])  # för A/B-experiment

city = rng.choice(cities, size=n, p=[0.28, 0.22, 0.40, 0.10])
gender = rng.choice(genders, size=n)
age = rng.integers(18, 65, size=n)

# A/B-tilldelning (balanced med lite slump)
group = rng.choice(groups, size=n)

# Aktivitet (ungefär som "engagement score")
base = rng.normal(60, 12, size=n)

# Små effekter från stad/kön
city_shift = {"Göteborg":2, "Malmö":-1, "Stockholm":1, "Uppsala":0}
gender_shift = {"F":1.5, "M":0}
score = base + np.vectorize(city_shift.get)(city) + np.vectorize(gender_shift.get)(gender)

# Effekt av grupp: B ger +2 i snitt
score = score + np.where(group=="B", 2.0, 0.0)

# Begränsa och runda
score = np.clip(score, 10, 100)

# Binärt utfall (t.ex. "köp"): sannolikhet ökar med score
p_buy = 1/(1 + np.exp(-(score-55)/10))
purchase = rng.binomial(1, p_buy)

# Intäkter för köp (lognormal-ish), 0 om inget köp
revenue = np.where(purchase==1, rng.lognormal(mean=3.2, sigma=0.5, size=n), 0.0)

# Lägg in lite saknade värden
df = pd.DataFrame({
    "city": city,
    "gender": gender,
    "age": age,
    "group": group,
    "score": np.round(score,1),
    "purchase": purchase,
    "revenue": np.round(revenue,2)
})
mask = rng.random(size=df.shape) < 0.01  # ~1% NaN
for (r,c),m in np.ndenumerate(mask):
    if m:
        df.iat[r,c] = np.nan

df.to_csv("week6_data.csv", index=False)
print("Skapade weekX_data.csv med", len(df), "rader.")

### Om datasetet (weekX_data.csv)
Ett **syntetiskt affärs/produkt-dataset** som passar både EDA och inferens.

| Kolumn | Typ | Beskrivning |
|---|---|---|
| `city` | kategori | Stad |
| `gender` | kategori | Kön (F/M) |
| `age` | int | Ålder |
| `group` | kategori | A/B-grupp |
| `score` | float | Aktivitet/engagemang 10–100 (högre ≈ mer aktiv) |
| `purchase` | 0/1 | Om kunden gjorde ett köp |
| `revenue` | float | Intäkt, 0 om inget köp |

**Noteringar:** Grupp **B** har liten positiv effekt på `score`. Sannolikheten för `purchase` ökar med `score`. Ca 1% saknade värden finns för att öva rengöring.

### Snabb kontroll
- Läs in filen till `df` och förhandsgranska.
- Kontrollera datatyper och antal saknade värden per kolumn.
- Skriv en kort kommentar (i kod) om något du reagerar på (t.ex. att `revenue` ofta är 0 p.g.a. icke-köp).

In [ ]:
import pandas as pd
df = pd.read_csv("week6_data.csv")

display(df.head())

df.info()

na_counts = df.isna().sum()
print("\nSaknade värden per kolumn:\n", na_counts)

# Kommentar: Om en "heltalskolumn" blivit float är det ofta p.g.a. NaN (heltal + NaN -> float).

## Block A

### Uppgift 1 – Tema, grid och etiketter
Rita en valfri figur över `score` (t.ex. histogram eller KDE) och förbättra läsbarheten:
- lägg till titel, axelrubriker, rutnät,
- justera tick-etiketter om det behövs,
- (tips: `plt.style.use(...)`, `plt.grid(True)`, `plt.xticks(rotation=...)`).

In [ ]:
import matplotlib.pyplot as plt

ax = df["score"].dropna().plot(kind="hist", bins=25)

ax.set_title("Fördelning av score")
ax.set_xlabel("score")
ax.set_ylabel("frekvens")
ax.grid(True, axis="y")
plt.tight_layout()
plt.show()

### Uppgift 2 – Flera paneler (subplots)
Gör två subplots bredvid varandra:
- till vänster: histogram av `score`,
- till höger: stapeldiagram över frekvens per `city`.
Se till att layouter inte överlappar (tips: `plt.tight_layout()`).

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 4))

df["score"].dropna().plot(kind="hist", bins=25, ax=axes[0])
axes[0].set_title("Score")
axes[0].set_xlabel("score")
axes[0].set_ylabel("frekvens")

df["city"].value_counts().plot(kind="bar", ax=axes[1])
axes[1].set_title("Frekvens per stad")
axes[1].set_xlabel("stad")
axes[1].set_ylabel("antal")
axes[1].grid(True, axis="y")

plt.tight_layout()
plt.show()

### Uppgift 3 – Färger och annoteringar
Rita en boxplot av `score` uppdelad på `group` och:
- färgsätt grupperna olika,
- lägg in en **horisontell referenslinje** vid `score = 60`,
- annotera grafen med en kort textpil som pekar på en intressant del.
*(tips: `ax.axhline(...)`, `ax.annotate(...)`)*

In [ ]:
data_A = df.loc[df["group"]=="A", "score"].dropna()
data_B = df.loc[df["group"]=="B", "score"].dropna()

fig, ax = plt.subplots()

# Boxplot med egna etiketter och möjlighet att färglägga lådor
bp = ax.boxplot([data_A, data_B], tick_labels=["A", "B"], patch_artist=True)

# Färglägg lådorna så grupperna blir tydliga
bp["boxes"][0].set_facecolor("tab:blue")
bp["boxes"][1].set_facecolor("tab:orange")

ax.axhline(60, color="gray", linestyle="--", linewidth=1)

# Annotera med en pil för att peka ut referenslinjen
ax.annotate("referens 60", xy=(1.5, 60), xytext=(1.5, 70),
            arrowprops=dict(arrowstyle="->"))

ax.set_title("Score per grupp")
ax.set_ylabel("score")
ax.grid(True, axis="y")
plt.tight_layout()
plt.show()

### Uppgift 4 – Felsökning (legend & labels)
Koden ritar två serier men legenden blir konstig/oläsbar. Kör och **rätta**:
```python
import pandas as pd, matplotlib.pyplot as plt
df = pd.read_csv("weekX_data.csv")
fig, ax = plt.subplots()
ax.plot(df.index, df["score"], "b.", label="score")
ax.plot(df.index, df["revenue"], "r.")   
ax.legend(loc="best")                    
plt.show()
```
**Mål:** gör legenden tydlig (rätt etiketter, ev. alpha/markersize) och se till att axlar har rubriker.

In [ ]:
import pandas as pd, matplotlib.pyplot as plt
df = pd.read_csv("week6_data.csv")
fig, ax = plt.subplots()
ax.plot(df.index, df["score"], "b.", label="score", alpha=0.5, markersize=4)
ax.plot(df.index, df["revenue"], "r.", label="revenue", alpha=0.5, markersize=4)   
ax.legend(loc="best")                    
plt.show()

### Uppgift 5 – Välj rätt figur
Du vill jämföra distributionen av `revenue` mellan `group` **för de som köpt** (`purchase==1`). 
- Välj en **lämplig** figurtyp (motivera kort i en kommentar).
- Rita grafen och gör den läsbar (titel, axlar, grid).

In [ ]:
# Filtrera till faktiska köpare (annars blir revenue = 0 en stor klump)
paid = df[df["purchase"] == 1].copy()

# Boxplot visar median/spridning/outliers tydligt per grupp
ax = paid.boxplot(column="revenue", by="group", grid=False)

# Snygga till etiketter
ax.set_title("Revenue (köpare) per grupp")
plt.suptitle("")  # ta bort överrubriken som boxplot by= lägger på
ax.set_xlabel("group")
ax.set_ylabel("revenue")
plt.grid(True, axis="y")
plt.tight_layout()
plt.show()

## Block B – EDA med Python

### Uppgift 6 – Grundprofil
- Räkna frekvens per `city` och `gender`.
- Räkna andel som köpt (`purchase`-medel fungerar här) per `city`.
- Skapa en tabell med medel/median för `score` och `revenue` per `group` och `gender`.
*(tips: `groupby(...).agg({...})`)*

In [ ]:
freq_city = df["city"].value_counts()
freq_gender = df["gender"].value_counts()
print("Frekvens per city:\n", freq_city)
print("\nFrekvens per gender:\n", freq_gender)

# Andel köpare per stad (mean av 0/1 ger andel)
share_purchase_city = df.groupby("city")["purchase"].mean().sort_values(ascending=False)
print("\nAndel köpare per city:\n", share_purchase_city)

# Centrala mått per (group, gender) för både score och revenue
summary = (df.groupby(["group","gender"])
             .agg(score_mean=("score","mean"),
                  score_median=("score","median"),
                  revenue_mean=("revenue","mean"),
                  revenue_median=("revenue","median")))
display(summary)

### Uppgift 7 – Samband mellan variabler
- Skapa ett **spridningsdiagram** med `score` (x) och `revenue` (y) **endast för köp**.
- Lägg gärna på en enkel trendlinje eller glidande medelvärde (valfritt).
- Kommentera om sambandet ser linjärt ut eller ej.

In [ ]:
paid = df[df["purchase"] == 1].copy()

# Rita scatter så vi ser spridningen
ax = paid.plot.scatter(x="score", y="revenue", alpha=0.5)
ax.set_title("Score vs Revenue (endast köpare)")
ax.set_xlabel("score")
ax.set_ylabel("revenue")
ax.grid(True)

# Lägg på glidande medelvärde för att antyda trend (sortera först på x)
paid = paid.sort_values("score")
rolling_mean = paid["revenue"].rolling(window=30, min_periods=10).mean()
plt.plot(paid["score"], rolling_mean, linewidth=2)

plt.tight_layout()
plt.show()

### Uppgift 8 – Rensa saknade värden
- Räkna saknade värden per kolumn.
- Fyll saknade i `age` med median; i `score` med medelvärde.
- Verifiera att saknade minskat.
*(tips: `fillna(...)`)*

In [ ]:
# Kolla läget före
print("Saknade före:\n", df.isna().sum())

# Fyll numeriska kolumner med enkla strategier
df["age"] = df["age"].fillna(df["age"].median())     # median=robust mot outliers
df["score"] = df["score"].fillna(df["score"].mean()) # medelvärde räcker i övning

# Stäm av att NaN minskat
print("\nSaknade efter:\n", df.isna().sum())

### Uppgift 9 – Felsökning (join/merge)
Koden ska slå ihop två tabeller men resultatet blir fler rader än väntat. Kör och **rätta** så att hopslagningen blir korrekt.
```python
import pandas as pd
left = pd.DataFrame({"city":["Göteborg","Malmö","Stockholm","Uppsala"], "kod":[1,2,3,4]})
right = pd.DataFrame({"stad":["Göteborg","Malmö","Stockholm","Uppsala"], "region":["Väst","Syd","Öst","Öst"]})
bad = left.merge(right)
```
**Mål:** använd rätt nycklar/kolumnnamn vid hopslagning och kontrollera resultatets storlek.

In [ ]:
left = pd.DataFrame({"city":["Göteborg","Malmö","Stockholm","Uppsala"], "kod":[1,2,3,4]})
right = pd.DataFrame({"stad":["Göteborg","Malmö","Stockholm","Uppsala"], "region":["Väst","Syd","Öst","Öst"]})
merged = left.merge(right, left_on="city", right_on="stad", how="left")

print("Antal rader left:", len(left), "| merged:", len(merged))
display(merged.head())

## Block C – Sampling och variation

### Uppgift 10 – Slumpmässigt stickprov
- Dra ett **enkelt slumpmässigt** stickprov på 100 rader ur `df`.
- Räkna medelvärde för `score` i stickprovet och jämför med hela datasetet.
- Upprepa 100 gånger och titta på variationen (tips: loop/lista/array).
*(tips: `df.sample(...)`)*

In [ ]:
# Populationsmedel (hela datan)
pop_mean = df["score"].mean()

# Repetera sampling för att se hur stickprovsmedlet varierar
means = []
for i in range(200):  # 200 upprepningar
    sample_mean = df.sample(n=100, replace=False, random_state=i)["score"].mean()
    means.append(sample_mean)

# Sammanfattning: populationsmedel, medel av stickprovsmedel och deras spridning
pop_mean, float(np.mean(means)), float(np.std(means, ddof=1))

### Uppgift 11 – Bootstrap av medelvärde
- Skapa 1000 bootstrap-stickprov från `score` (med återläggning).
- Beräkna bootstrap-fördelningen av medelvärdet och visa ett histogram.
- Skriv ut bootstrap-medel och -standardavvikelse.
*(tips: `rng.integers(0, n, size=...)` för indexsampling)*

In [ ]:
x = df["score"].dropna().to_numpy()

rng = np.random.default_rng(0)  # frö för reproducerbarhet
n = x.size

# Bygg bootstrap-fördelning för medelvärde
means = np.array([x[rng.integers(0, n, size=n)].mean() for _ in range(1000)])

print("bootstrap mean:", float(means.mean()), "bootstrap std:", float(means.std(ddof=1)))

# Visa fördelningen
plt.hist(means, bins=30)
plt.title("Bootstrap-fördelning för mean(score)")
plt.xlabel("mean(score)")
plt.ylabel("frekvens")
plt.grid(True)
plt.show()

### Uppgift 12 – Felsökning
Koden försöker skapa reproducerbar slump men fungerar inte som tänkt. Kör och **rätta**.
```python
import numpy as np, pandas as pd
df = pd.read_csv("weekX_data.csv")
np.random.seed(42)
a = np.random.rand(3)
rng = np.random.default_rng(42)
b = rng.random(3)
c = rng.random(3)
print(a, b, c)    # förväntar sig samma b,c varje körning men får olika i vissa miljöer
```
**Mål:** använd **en** slumpgenerator konsekvent och påpeka varför mix av API:er kan förvirra.

In [ ]:
# Skapa EN RNG och använd den genomgående
rng = np.random.default_rng(42)

# Alla slumptal i scriptet kommer nu från samma RNG/seed
a = rng.random(3)  # första sekvensen
b = rng.random(3)  # fortsättning på samma sekvens
c = rng.random(3)  # fortsättning

print("a:", a)
print("b:", b)
print("c:", c)  # samma värden varje körning med samma seed

## Block D – Konfidensintervall

### Uppgift 13 – CI för medelvärde (känd sigma antas ok)
- Skatta 95% konfidensintervall för medelvärdet av `score` i **grupp A**.
- Anta stort n så att normalapprox är rimlig.
- Redovisa punktestimat, standardfel, under- och övre gräns.
*(tips: `z ≈ 1.96` vid 95%)*

In [ ]:
# Plocka ut serien vi vill skapa CI för (grupp A)
x = df.loc[df["group"]=="A", "score"].dropna().to_numpy()

# Beräkna punktestimat och osäkerhet
mean = x.mean()                 # punktestimat
sd = x.std(ddof=1)              # stickprovs-SD
n = x.size
se = sd / np.sqrt(n)            # standardfel

# 95% normalapprox (vid stort n)
z = 1.96
ci = (mean - z*se, mean + z*se)

mean, se, ci

### Uppgift 14 – CI för andel
- Skatta 95% konfidensintervall för andelen `purchase==1` i **grupp B**.
- Använd en standardformel för andelars CI.
- Redovisa punktestimat (`p̂`), standardfel och gränser.

In [ ]:
# Plocka ut köp (0/1) för grupp B
x = df.loc[df["group"]=="B", "purchase"].dropna().to_numpy()

p_hat = x.mean()                # punktestimat (andel)
n = x.size
se = np.sqrt(p_hat*(1 - p_hat) / n)  # standardfel för andel

z = 1.96
ci = (p_hat - z*se, p_hat + z*se)

p_hat, se, ci

### Uppgift 15 – Felsökning (fel standardfel)
Någon har blandat ihop **standardavvikelse** och **standardfel** i ett CI för medel.
Korrigera formeln och beräkna om.
```python
import numpy as np, pandas as pd
df = pd.read_csv("weekX_data.csv")
x = df.loc[df["group"]=="A", "score"].dropna()
mean = x.mean()
sd = x.std()
n = x.shape[0]
z = 1.96
lo = mean - z*sd
hi = mean + z*sd
(lo, hi)
```
**Mål:** beräkna **standardfel** korrekt och rapportera nytt intervall.

In [ ]:
x = df.loc[df["group"]=="A", "score"].dropna()

mean = x.mean()
sd = x.std(ddof=1)
n = x.shape[0]

# standardfel (SE) istället för SD
se = sd / np.sqrt(n)

z = 1.96
ci = (mean - z*se, mean + z*se)
ci

## Block E – Hypotesprövning

### Uppgift 16 – A/B-test av medelvärden
- Testa om `score` skiljer sig mellan **grupp A** och **grupp B**.
- Formulera noll- och mothypotes, välj lämpligt test, rapportera p-värde.
- Kommentera kort hur du tolkar resultatet (praktisk betydelse vs statistisk).
*(tips: tänk oberoende stickprov och medelvärden)*

In [ ]:
# Dela upp score i A och B
x = df.loc[df["group"]=="A", "score"].dropna().to_numpy()
y = df.loc[df["group"]=="B", "score"].dropna().to_numpy()

# Gruppvisa medel och varianser
mx, my = x.mean(), y.mean()
vx, vy = x.var(ddof=1), y.var(ddof=1)
nx, ny = x.size, y.size

# Standardfel för skillnaden i medel (Welch)
se_diff = np.sqrt(vx/nx + vy/ny)

# t-statistik (utan lika varians-antagande)
t_stat = (mx - my) / se_diff

(mx, my), se_diff, t_stat  # p-värde kräver t-fördelningen (t.ex. via SciPy)

### Uppgift 17 – Test av andelar
- Testa om andelen `purchase==1` skiljer sig mellan **grupp A** och **grupp B**.
- Skriv upp teststatistika och p-värde.
- Kommentera kort om antagandena verkar rimliga för ditt valda test.

In [ ]:
# Binär utfallsvariabel: purchase
a = df.loc[df["group"]=="A","purchase"].dropna().to_numpy()
b = df.loc[df["group"]=="B","purchase"].dropna().to_numpy()

p1, p2 = a.mean(), b.mean()
n1, n2 = a.size, b.size

# Poolad andel (antar H0: p1 = p2)
p_pool = (a.sum() + b.sum()) / (n1 + n2)

# Standardfel enligt poolad andel
se = np.sqrt(p_pool*(1 - p_pool)*(1/n1 + 1/n2))

# z-statistik för skillnaden i andelar
z_stat = (p1 - p2) / se

(p1, p2), se, z_stat

### Uppgift 18 – Felsökning (p-värde-fälla)
Koden drar fel slutsats. Kör och **rätta** resonemanget.
```python
p_value = 0.07
if p_value < 0.1:
    print("Signifikant på 5%-nivån!")  # FEL slutsats
```
**Mål:** tolka p-värden korrekt och knyta till vald signifikansnivå (t.ex. 5%).

In [ ]:
alpha = 0.05      # vald signifikansnivå (5%)
p_value = 0.07    # givet p-värde i exempel

# Rätt tolkning: kräver p < alpha för 5%-nivån
if p_value < alpha:
    print("Signifikant på 5%-nivån.")
else:
    print(f"Inte signifikant på 5%-nivån (p = {p_value}).")